# Building Deep Learning Applications with Keras: Deploying your model using a REST API


Once we have trained a Deep Learning model and verifyed that is generalizing well, it is time to put our model into production. By production, this means that other applications or even real users will now be able to interact with this model in real time. Therefore, an user or an application will send data to our model and expect a prediction as a response. Most of the time, our model will be hosted in a server that will them receive a request and return a answer for this request. Therefore, to other applications or users to communicate with our model, we need to define its API.

Now let's do this for an image classification model. Our model will receive an image and output which object it thinks that is present in that image. Therefore, we need to define 4 things for this model to handle requests:

* Load the model in memory
* Define how the model will receive a request containing an image
* Pre-process the image to allow it to be used on the model
* Define how the model prediction will be returned to the user.

Let's start by first loading the model in memory. We can define the following function:

In [ ]:
from keras.applications import ResNet50

def load_model():
    global model
    model = ResNet50(weights="imagenet")


Before defining the model API, we know that it will receive an image and that our model only accepts images with a certain format to it. Let's define a function that receives an image and pre-process it for our model.

In [ ]:
from keras.preprocessing.image import img_to_array
from keras.applications import imagenet_utils
from PIL import Image
import numpy as np

def pre_process_image(image, resize_size):
    if image.mode != "RGB":
        image = image.convert("RGB")

    image = image.resize(resize_size)
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = imagenet_utils.preprocess_input(image)
    return image

If this was text data, in this step we would convert our text into a list of ids, just as we seen on the LSTM notebook. This is very important step to remember. Every model that you implement and want to put it in production will receive unformarted data. Therefore, **always remember to pre-process this data in order to not crash the model.**

Once we have loaded the model in memory and defined a pre-processing function for the data it receives, we can now create an API. To create this API, we will use [Flask](http://flask.pocoo.org/). This python application will allow us to handle HTTP requests.

Let's now initiate our Flask application:

In [ ]:
import flask
app = flask.Flask(__name__)

Flask will also allow us to define an **end point** to our model.

What do I mean by an **end point** ? We understand that our model will be running on a server. That server will have an address, let's say it is running on the address **0.0.0.0:8000**. Inside this address, we need to define how we are going to access our model. We need to identify our model as **resource** and give it an unique name. For example, we can define that our model will be found on the following name:

**0.0.0.0:8000/model/**

Now, our application will know that requests for this address will be sent to our model.

Furthermore, we need to define which type of HTTP request our model will answer to. Since we will receive data to classify, we will need to define that the model will answer to [POST](https://en.wikipedia.org/wiki/POST_(HTTP) requests. 

Finally, we need to define how the model will send the prediction back to the user. This means that we must convert our prediction into a format that other applications will be able to parse. One of the most widely used formats for this task is the [json](https://www.json.org/json-en.html) format. This is a simple format that states that every data will be stored via a key-value pair. For example, suppose we want to inform for an user that the photo he has uploaded has a dog in it and the level of certainty our model has in that prediction. We can define our json response as follows:

In [ ]:
{
    "prediction": "dog",
    "certainty": 0.98
}

We can see that we have pairs of variables in the json format, where the **key** identifies the variable and the **value**, obviously states the value of the key variable. It is very important for your application to define how it will send its reponse back to applications, so that these applications can act accordingly.

With that said, let's define a function that handle **POST** request and sends a response back to the application that made the request. Our function will need to:

* Extract the image from the request it has received
* Pre-process the image
* Use the model to make a prediction
* Create a json file for that prediction and return that json file

In [ ]:
import io


"""
This decorator creates our end point. It states that our model will be found on the "model" address
and that it will handle POST request.
"""
@app.route("/model", methods=["POST"])
def predict():
    data = {"success": False}

    if flask.request.method == "POST":
            
        """
        First let's extract the image from the request
        """
        image = flask.request.files["image"].read()
        image = Image.open(io.BytesIO(image))

        """
        Now, let's pre-process the image
        """
        image = pre_process_image(image, resize_size=(224, 224))
            
        """
        Let's perform the classification over this image using our model
        """
        preds = model.predict(image)
        results = imagenet_utils.decode_predictions(preds)
            
        """
        Finally, let's create our json response and return it to the application that
        sent the request.
        """
            
        data["predictions"] = []

        # loop over the results and add them to the list of
        # returned predictions
        for (imagenetID, label, prob) in results[0]:
            r = {"label": label, "probability": float(prob)}
            data["predictions"].append(r)

        data["success"] = True

    return flask.jsonify(data)

With that function defined, let's test it. We will use the following image to test our model:

<img src="images/dog.jpg">

Let's run our application. We just need to run the following code:

In [ ]:
from multiprocessing import Process
from werkzeug.serving import run_simple

def run_server():
    print("Loading model")
    load_model()
    print("Running server")
    run_simple('0.0.0.0', 9000, app)

p = Process(target=run_server)
p.start()

Now, let's send the dog image to our model and test if it is working properly. To do that, run the following **curl** command.

In [ ]:
! curl -X POST -F image=@images/dog.jpg 'http://localhost:9000/model'

We expect that our API should return something similar to this json response:

{
    "predictions":
        [
            {"label":"beagle","probability":0.9901767373085022},
            {"label":"Walker_hound","probability":0.002248708624392748},
            {"label":"Brittany_spaniel","probability":0.0011901347897946835},
            {"label":"pot","probability":0.001180286519229412},
            {"label":"Cardigan","probability":0.0006831097998656332}
        ],
    "success":true
}

If you see a similar response, then your model is running correctly.
    
If you are interested in model deployment, this is just the first step on that area. You can check more advanced ways to deploy your model, for example, how can I scale my model ? Should I use containers ?

You can start learning about this topics reading this [post](https://www.pyimagesearch.com/2018/01/29/scalable-keras-deep-learning-rest-api/) and learning more about [docker](https://www.docker.com/). Model deployment is becoming more and more present when dealing with Deep Learning models and is always good to keep learning new techniques to handle that activity :)